In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Tea and coffee from 2015 to 2020

In [52]:
tea_coffee=pd.read_csv('FAOS_Coffee_Tea_2-11-2023.csv')

In [53]:
region=pd.read_csv('RegionCountry.csv')

### Data preparation tea_coffee

In [92]:
tea_coffee.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,Food Balances (2010-),4,Afghanistan,5611,Import Quantity,S2630,Coffee and products,2015,2015,1000 tonnes,0,I,Imputed value
1,FBS,Food Balances (2010-),4,Afghanistan,5611,Import Quantity,S2630,Coffee and products,2016,2016,1000 tonnes,0,I,Imputed value
2,FBS,Food Balances (2010-),4,Afghanistan,5611,Import Quantity,S2630,Coffee and products,2017,2017,1000 tonnes,2,I,Imputed value
3,FBS,Food Balances (2010-),4,Afghanistan,5611,Import Quantity,S2630,Coffee and products,2018,2018,1000 tonnes,1,I,Imputed value
4,FBS,Food Balances (2010-),4,Afghanistan,5611,Import Quantity,S2630,Coffee and products,2019,2019,1000 tonnes,1,I,Imputed value


checking values for each columns

In [55]:
tea_coffee['Domain Code'].unique()

array(['FBS'], dtype=object)

In [56]:
tea_coffee['Domain'].unique()

array(['Food Balances (2010-)'], dtype=object)

In [57]:
tea_coffee['Element'].unique()

array(['Import Quantity', 'Export Quantity', 'Domestic supply quantity',
       'Production', 'Tourist consumption'], dtype=object)

In [58]:
tea_coffee['Item'].unique()

array(['Coffee and products', 'Tea (including mate)'], dtype=object)

In [59]:
tea_coffee['Year'].unique()

array([2015, 2016, 2017, 2018, 2019, 2020], dtype=int64)

In [60]:
tea_coffee['Unit'].unique()

array(['1000 tonnes'], dtype=object)

In [61]:
tea_coffee['Flag'].unique()

array(['I', 'E'], dtype=object)

In [62]:
tea_coffee['Flag Description'].unique()

array(['Imputed value', 'Estimated value'], dtype=object)

#### Delete unusable columns

In [64]:
tea_coffe_20152020=tea_coffee.drop(['Domain Code','Domain','Unit','Flag','Flag Description','Year Code','Item Code (CPC)','Element Code','Area Code (M49)'],axis=1)
tea_coffe_20152020.head()

,Area,Element,Item,Year,Value
0,Afghanistan,Import Quantity,Coffee and products,2015,0
1,Afghanistan,Import Quantity,Coffee and products,2016,0
2,Afghanistan,Import Quantity,Coffee and products,2017,2
3,Afghanistan,Import Quantity,Coffee and products,2018,1
4,Afghanistan,Import Quantity,Coffee and products,2019,1


In [20]:
tea_coffe_20152020.shape

(7856, 5)

In [21]:
tea_coffe_20152020.dtypes

Area       object
Element    object
Item       object
Year        int64
Value       int64
dtype: object

In [22]:
tea_coffe_20152020.isnull().sum()

Area       0
Element    0
Item       0
Year       0
Value      0
dtype: int64

In [23]:
tea_coffe_20152020.duplicated().sum()

0

#### Pivot table with items in 'element' column as column names

In [65]:
pivot_Tea_coffee = pd.pivot_table(tea_coffe_20152020, index=['Area', 'Item', 'Year'], columns='Element', values='Value').reset_index()

In [67]:
pivot_Tea_coffee.head()

Element,Area,Item,Year,Domestic supply quantity,Export Quantity,Import Quantity,Production,Tourist consumption
0,Afghanistan,Coffee and products,2015,0.0,0.0,0.0,NaN,NaN
1,Afghanistan,Coffee and products,2016,0.0,0.0,0.0,NaN,NaN
2,Afghanistan,Coffee and products,2017,0.0,0.0,2.0,NaN,NaN
3,Afghanistan,Coffee and products,2018,0.0,1.0,1.0,NaN,NaN
4,Afghanistan,Coffee and products,2019,0.0,0.0,1.0,NaN,NaN


In [23]:
pivot_Tea_coffee.shape

(2184, 8)

In [71]:
pivot_Tea_coffee.isnull().sum()

Element
Area                           0
Item                           0
Year                           0
Domestic supply quantity       0
Export Quantity              173
Import Quantity                2
Production                  1111
Tourist consumption         1778
dtype: int64

Because I have too little observation in the column "Tourist consumption"

In [25]:
pivot_Tea_coffee=pivot_Tea_coffee.drop('Tourist consumption',axis=1)

#### I decide to split into 2 dataframes because Production has too many nulls and I need to delete those rows without droping those rows in the main dataframe

In [ ]:
pivot_Tea_coffee = pivot_Tea_coffee.loc[:, ['Area', 'Item', 'Year', 'Domestic supply quantity', 'Export Quantity', 'Import Quantity']]
production_tea_coffee = pivot_Tea_coffee.loc[:, ['Area', 'Item', 'Year', 'Production']]

In [76]:
production_tea_coffee=production_tea_coffee[~production_tea_coffee["Production"].isna()]
production_tea_coffee.head(10)

Element,Area,Item,Year,Production
12,Albania,Coffee and products,2015,0.0
13,Albania,Coffee and products,2016,0.0
14,Albania,Coffee and products,2017,0.0
15,Albania,Coffee and products,2018,0.0
16,Albania,Coffee and products,2019,0.0
17,Albania,Coffee and products,2020,0.0
36,Angola,Coffee and products,2015,15.0
37,Angola,Coffee and products,2016,15.0
38,Angola,Coffee and products,2017,15.0
39,Angola,Coffee and products,2018,16.0


In [80]:
pivot_Tea_coffee.head()

Element,Area,Item,Year,Domestic supply quantity,Export Quantity,Import Quantity
0,Afghanistan,Coffee and products,2015,0.0,0.0,0.0
1,Afghanistan,Coffee and products,2016,0.0,0.0,0.0
2,Afghanistan,Coffee and products,2017,0.0,0.0,2.0
3,Afghanistan,Coffee and products,2018,0.0,1.0,1.0
4,Afghanistan,Coffee and products,2019,0.0,0.0,1.0


Let's now check the countries with no values in export and then in import

In [82]:
pivot_Tea_coffee[pivot_Tea_coffee["Export Quantity"].isna()]

Element,Area,Item,Year,Domestic supply quantity,Export Quantity,Import Quantity
18,Albania,Tea (including mate),2015,0.0,NaN,0.0
21,Albania,Tea (including mate),2018,0.0,NaN,0.0
53,Antigua and Barbuda,Coffee and products,2020,0.0,NaN,0.0
56,Antigua and Barbuda,Tea (including mate),2017,0.0,NaN,0.0
57,Antigua and Barbuda,Tea (including mate),2018,0.0,NaN,0.0
108,Azerbaijan,Coffee and products,2015,1.0,NaN,1.0
114,Azerbaijan,Tea (including mate),2015,2.0,NaN,12.0
116,Azerbaijan,Tea (including mate),2017,10.0,NaN,14.0
120,Bahamas,Coffee and products,2015,1.0,NaN,1.0
126,Bahamas,Tea (including mate),2015,1.0,NaN,1.0


In [83]:
pivot_Tea_coffee[pivot_Tea_coffee["Import Quantity"].isna()]

Element,Area,Item,Year,Domestic supply quantity,Export Quantity,Import Quantity
350,Central African Republic,Tea (including mate),2015,0.0,NaN,NaN
352,Central African Republic,Tea (including mate),2017,0.0,NaN,NaN


I decide to delete those rows for the analysis

In [ ]:
pivot_Tea_coffee.dropna()

### Data preparation region

In [89]:
region.head(15)

,REGION (DISPLAY),COUNTRY (DISPLAY)
0,Eastern Mediterranean,Afghanistan
1,Europe,Albania
2,Africa,Algeria
3,Africa,Angola
4,Americas,Antigua and Barbuda
5,Americas,Argentina
6,Europe,Armenia
7,Western Pacific,Australia
8,Europe,Austria
9,Europe,Azerbaijan


In [91]:
region.shape

(179, 2)

In [85]:
region.dtypes

REGION (DISPLAY)     object
COUNTRY (DISPLAY)    object
dtype: object

In [86]:
region.isnull().sum()

REGION (DISPLAY)     0
COUNTRY (DISPLAY)    0
dtype: int64

In [87]:
region.duplicated().sum()

0

#### Export the datas

In [34]:
pivot_Tea_coffee.to_csv('Tea_coffee_20152020.csv', index=False)

In [35]:
production_tea_coffee.to_csv'Production_tea_coffe_20152020.csv',index=False)